# Inner Join VCF with Ensembl Chromosome 1 Data

This notebook demonstrates joining VCF variants with Ensembl variation data for Ensembl

In [1]:
import polars as pl
import polars_bio as pb
from pathlib import Path
from platformdirs import user_cache_dir


In [2]:
# Configure Polars to show more rows and columns
pl.Config.set_tbl_rows(-1)  # Show all rows
pl.Config.set_tbl_cols(-1)  # Show all columns
pl.Config.set_tbl_width_chars(1000)  # Increase table width
pl.Config.set_fmt_str_lengths(1000)  # Show longer string values without truncation


polars.config.Config

In [3]:
# Set up paths
current_folder = Path.cwd().absolute().resolve()
if current_folder.name == "notebooks":
    current_folder = current_folder.parent

vcf_path = current_folder / "data" / "tests" / "antku_small.vcf"
cache_dir = Path(user_cache_dir(appname="genobear")) / "ensembl_variations" / "splitted_variants" / "SNV"
chr1_path = cache_dir / "homo_sapiens-chr1.parquet"


In [4]:
# Load VCF file using polars-bio
vcf_lazy = pb.scan_vcf(str(vcf_path))
print(f"VCF file: {vcf_path}")
print(f"VCF schema: {vcf_lazy.collect_schema()}")


VCF file: /home/antonkulaga/sources/genobear/data/tests/antku_small.vcf
VCF schema: Schema({'chrom': String, 'start': UInt32, 'end': UInt32, 'id': String, 'ref': String, 'alt': String, 'qual': Float64, 'filter': String, 'END': Int32})


In [10]:
import polars as pl

def is_sorted_by_chrom_start(vcf_lazy: pl.LazyFrame) -> bool:
    chrom_txt = pl.col("chrom").cast(pl.Utf8).str.replace(r"^chr", "", literal=False)
    chrom_key = (
        pl.when(chrom_txt.str.to_uppercase() == "X").then(23)
        .when(chrom_txt.str.to_uppercase() == "Y").then(24)
        .when(chrom_txt.str.to_uppercase().is_in(["MT", "M"])).then(25)
        .otherwise(chrom_txt.cast(pl.Int32, strict=False))
        .alias("_chrom_key")
    )

    cur_chrom = chrom_key
    nxt_chrom = cur_chrom.shift(-1)
    cur_start = pl.col("start")
    nxt_start = cur_start.shift(-1)

    violation = (nxt_chrom < cur_chrom) | ((nxt_chrom == cur_chrom) & (nxt_start < cur_start))

    # Use Expr-native aggregation to keep type checkers happy.
    is_sorted_expr = violation.fill_null(False).any().not_()

    return bool(
        vcf_lazy.select(is_sorted_expr.alias("is_sorted"))
        .collect(streaming=True)
        .item()
    )

In [11]:
is_sorted_by_chrom_start(vcf_lazy)

/tmp/ipykernel_367571/2559437693.py:25: DeprecationWarning: the `streaming` parameter was deprecated in 1.25.0; use `engine` instead.
  .collect(streaming=True)


0rows [00:00, ?rows/s]

True

In [5]:
#special cell
from genobear.annotation.ensembl_annotations import download_ensembl_annotations
import polars as pl

# Get the Ensembl cache path and scan SNV parquets
ensembl_cache = download_ensembl_annotations()
snv_dir = ensembl_cache / "data" / "SNV"
if not snv_dir.exists():
    snv_dir = ensembl_cache / "SNV"

# Scan all parquet files in the directory as one lazy frame
df_ensembl = pl.scan_parquet(snv_dir)

In [6]:
df_ensembl.head(10).collect()

chrom,start,end,id,ref,alt,qual,filter,cosmic_101,clinvar_202502,dbsnp_156,hgmd-public_20204,tsa,e_cited,e_multiple_observations,e_freq,e_topmed,e_hapmap,e_phenotype_or_disease,e_esp,e_gnomad,e_1000g,e_exac,clin_risk_factor,clin_protective,clin_confers_sensitivity,clin_other,clin_drug_response,clin_uncertain_significance,clin_benign,clin_likely_pathogenic,clin_pathogenic,clin_likely_benign,clin_histocompatibility,clin_not_provided,clin_association,ma,maf,mac,aa
str,u32,u32,str,str,str,f64,str,bool,bool,bool,bool,str,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,str,f32,i32,str
"""1""",10001,10001,"""rs1570391677""","""T""","""A""",null,"""""",false,false,true,false,"""SNV""",false,false,true,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,null,null,null,null
"""1""",10001,10001,"""rs1570391677""","""T""","""C""",null,"""""",false,false,true,false,"""SNV""",false,false,true,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,null,null,null,null
"""1""",10002,10002,"""rs1570391692""","""A""","""C""",null,"""""",false,false,true,false,"""SNV""",false,false,true,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,null,null,null,null
"""1""",10003,10003,"""rs1570391694""","""A""","""C""",null,"""""",false,false,true,false,"""SNV""",false,false,true,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,null,null,null,null
"""1""",10007,10007,"""rs1639538116""","""T""","""C""",null,"""""",false,false,true,false,"""SNV""",false,false,true,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,null,null,null,null
"""1""",10007,10007,"""rs1639538116""","""T""","""G""",null,"""""",false,false,true,false,"""SNV""",false,false,true,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,null,null,null,null
"""1""",10008,10008,"""rs1570391698""","""A""","""C""",null,"""""",false,false,true,false,"""SNV""",false,false,true,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,null,null,null,null
"""1""",10008,10008,"""rs1570391698""","""A""","""G""",null,"""""",false,false,true,false,"""SNV""",false,false,true,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,null,null,null,null
"""1""",10008,10008,"""rs1570391698""","""A""","""T""",null,"""""",false,false,true,false,"""SNV""",false,false,true,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,null,null,null,null


In [7]:
df_ensembl.head(10).collect()

In [5]:
# Load chromosome 1 from Ensembl cache
chr1_lazy = pl.scan_parquet(str(chr1_path))
print(f"Chromosome 1 file: {chr1_path}")
print(f"Chromosome 1 schema: {chr1_lazy.collect_schema()}")


Chromosome 1 file: /home/antonkulaga/.cache/genobear/ensembl_variations/splitted_variants/SNV/homo_sapiens-chr1.parquet
Chromosome 1 schema: Schema({'chrom': String, 'start': UInt32, 'end': UInt32, 'id': String, 'ref': String, 'alt': String, 'qual': Float64, 'filter': String, 'cosmic_101': Boolean, 'clinvar_202502': Boolean, 'dbsnp_156': Boolean, 'hgmd-public_20204': Boolean, 'tsa': String, 'e_cited': Boolean, 'e_multiple_observations': Boolean, 'e_freq': Boolean, 'e_topmed': Boolean, 'e_hapmap': Boolean, 'e_phenotype_or_disease': Boolean, 'e_esp': Boolean, 'e_gnomad': Boolean, 'e_1000g': Boolean, 'e_exac': Boolean, 'clin_risk_factor': Boolean, 'clin_protective': Boolean, 'clin_confers_sensitivity': Boolean, 'clin_other': Boolean, 'clin_drug_response': Boolean, 'clin_uncertain_significance': Boolean, 'clin_benign': Boolean, 'clin_likely_pathogenic': Boolean, 'clin_pathogenic': Boolean, 'clin_likely_benign': Boolean, 'clin_histocompatibility': Boolean, 'clin_not_provided': Boolean, 'cli

In [7]:
# Filter VCF to chromosome 1 and perform inner join
vcf_chr1: pl.LazyFrame = vcf_lazy.filter(pl.col("chrom") == "1")

join_columns = ["chrom", "start", "ref", "alt"]
joined = vcf_chr1.join(
    chr1_lazy,
    on=join_columns,
    how="inner",
    suffix="_ensembl"
)

print("Joined schema:")
print(joined.collect_schema())


Joined schema:
Schema({'chrom': String, 'start': UInt32, 'end': UInt32, 'id': String, 'ref': String, 'alt': String, 'qual': Float64, 'filter': String, 'END': Int32, 'end_ensembl': UInt32, 'id_ensembl': String, 'qual_ensembl': Float64, 'filter_ensembl': String, 'cosmic_101': Boolean, 'clinvar_202502': Boolean, 'dbsnp_156': Boolean, 'hgmd-public_20204': Boolean, 'tsa': String, 'e_cited': Boolean, 'e_multiple_observations': Boolean, 'e_freq': Boolean, 'e_topmed': Boolean, 'e_hapmap': Boolean, 'e_phenotype_or_disease': Boolean, 'e_esp': Boolean, 'e_gnomad': Boolean, 'e_1000g': Boolean, 'e_exac': Boolean, 'clin_risk_factor': Boolean, 'clin_protective': Boolean, 'clin_confers_sensitivity': Boolean, 'clin_other': Boolean, 'clin_drug_response': Boolean, 'clin_uncertain_significance': Boolean, 'clin_benign': Boolean, 'clin_likely_pathogenic': Boolean, 'clin_pathogenic': Boolean, 'clin_likely_benign': Boolean, 'clin_histocompatibility': Boolean, 'clin_not_provided': Boolean, 'clin_association': 

In [7]:
# Show joined results
joined.head(20).collect()


0rows [00:00, ?rows/s]

chrom,start,end,id,ref,alt,qual,filter,END,end_ensembl,id_ensembl,qual_ensembl,filter_ensembl,cosmic_101,clinvar_202502,dbsnp_156,hgmd-public_20204,tsa,e_cited,e_multiple_observations,e_freq,e_topmed,e_hapmap,e_phenotype_or_disease,e_esp,e_gnomad,e_1000g,e_exac,clin_risk_factor,clin_protective,clin_confers_sensitivity,clin_other,clin_drug_response,clin_uncertain_significance,clin_benign,clin_likely_pathogenic,clin_pathogenic,clin_likely_benign,clin_histocompatibility,clin_not_provided,clin_association,ma,maf,mac,aa
str,u32,u32,str,str,str,f64,str,i32,u32,str,f64,str,bool,bool,bool,bool,str,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,str,f32,i32,str
"""1""",10015,10015,"""""","""A""","""G""",0.0,"""RefCall""",null,10015,"""rs1570391706""",null,"""""",false,false,true,false,"""SNV""",false,false,true,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,null,null,null,null
"""1""",10021,10021,"""""","""A""","""G""",0.0,"""RefCall""",null,10021,"""rs1570391710""",null,"""""",false,false,true,false,"""SNV""",false,false,true,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,null,null,null,null
"""1""",10027,10027,"""""","""A""","""G""",0.0,"""RefCall""",null,10027,"""rs1570391716""",null,"""""",false,false,true,false,"""SNV""",false,false,true,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,null,null,null,null
"""1""",10033,10033,"""""","""A""","""G""",0.0,"""RefCall""",null,10033,"""rs1570391722""",null,"""""",false,false,true,false,"""SNV""",false,false,true,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,null,null,null,null
"""1""",10039,10039,"""""","""A""","""G""",0.0,"""RefCall""",null,10039,"""rs978760828""",null,"""""",false,false,true,false,"""SNV""",false,false,true,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,null,null,null,null
"""1""",10045,10045,"""""","""A""","""G""",0.0,"""RefCall""",null,10045,"""rs1570391729""",null,"""""",false,false,true,false,"""SNV""",false,false,true,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,null,null,null,null
"""1""",10051,10051,"""""","""A""","""G""",0.0,"""RefCall""",null,10051,"""rs1052373574""",null,"""""",false,false,true,false,"""SNV""",false,false,true,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,null,null,null,null
"""1""",10250,10250,"""""","""A""","""C""",0.0,"""RefCall""",null,10250,"""rs199706086""",null,"""""",false,false,true,false,"""SNV""",false,false,true,false,false,false,false,true,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,null,null,null,null
"""1""",10257,10257,"""""","""A""","""C""",0.0,"""RefCall""",null,10257,"""rs111200574""",null,"""""",false,false,true,false,"""SNV""",false,false,true,false,false,false,false,true,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,null,null,null,null


In [8]:
# Statistics
total_vcf_chr1 = vcf_chr1.select(pl.count()).collect().item()
total_joined = joined.select(pl.count()).collect().item()
print(f"Total VCF variants on chr1: {total_vcf_chr1}")
print(f"Variants with Ensembl matches: {total_joined}")
print(f"Match rate: {total_joined / total_vcf_chr1 * 100:.2f}%")


/tmp/ipykernel_535970/1024664876.py:2: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)
  total_vcf_chr1 = vcf_chr1.select(pl.count()).collect().item()


0rows [00:00, ?rows/s]

/tmp/ipykernel_535970/1024664876.py:3: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)
  total_joined = joined.select(pl.count()).collect().item()


0rows [00:00, ?rows/s]

Total VCF variants on chr1: 793
Variants with Ensembl matches: 639
Match rate: 80.58%
